# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [125]:
housing = pd.read_csv('cal_housing_clean.csv')

In [126]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [127]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [128]:
x_data = housing.drop('medianHouseValue', axis = 1, )

In [129]:
x_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [130]:
y_true = housing['medianHouseValue']

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_true, test_size = 0.3, random_state = 101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [133]:
from sklearn.preprocessing import MinMaxScaler

In [134]:
sc =MinMaxScaler(feature_range= (0,1))

In [135]:
sc.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [136]:
X_train = pd.DataFrame(sc.fit_transform(X_train), columns= X_train.columns, index= X_train.index)

In [137]:
X_test = pd.DataFrame(sc.fit_transform(X_test), columns= X_test.columns, index= X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [138]:
x_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome'],
      dtype='object')

All the features are **Continuous Features**

In [139]:
housingAge = tf.feature_column.numeric_column('housingMedianAge')
totalRooms = tf.feature_column.numeric_column('totalRooms')
totalBedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
medianIncome = tf.feature_column.numeric_column('medianIncome')

In [140]:
feat_columns = [housingAge, totalBedrooms, totalRooms, population, households, medianIncome]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [141]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train, y_train, batch_size= 10, num_epochs= 1000, shuffle= True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [142]:
dnn_model = tf.estimator.DNNRegressor(hidden_units= [10,10,10], feature_columns= feat_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_num_ps_replicas': 0, '_task_id': 0, '_keep_checkpoint_max': 5, '_master': '', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027BB77A2588>, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\prash\\AppData\\Local\\Temp\\tmpjbvo2fl4', '_tf_random_seed': None, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_session_config': None, '_task_type': 'worker', '_service': None, '_save_checkpoints_steps': None, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [143]:
dnn_model.train(input_fn= input_func, steps = 15000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\prash\AppData\Local\Temp\tmpjbvo2fl4\model.ckpt.
INFO:tensorflow:step = 0, loss = 592823840000.0
INFO:tensorflow:global_step/sec: 56.2485
INFO:tensorflow:step = 100, loss = 452586300000.0 (1.778 sec)
INFO:tensorflow:global_step/sec: 138.677
INFO:tensorflow:step = 200, loss = 749598900000.0 (0.725 sec)
INFO:tensorflow:global_step/sec: 142.721
INFO:tensorflow:step = 300, loss = 346347440000.0 (0.697 sec)
INFO:tensorflow:global_step/sec: 133.034
INFO:tensorflow:step = 400, loss = 305627400000.0 (0.752 sec)
INFO:tensorflow:global_step/sec: 129.47
INFO:tensorflow:step = 500, loss = 223672810000.0 (0.772 sec)
INFO:tensorflow:global_step/sec: 155.556
INFO:tensorflow:step = 600, loss = 400636970000.0 (0.643

INFO:tensorflow:global_step/sec: 182.352
INFO:tensorflow:step = 7700, loss = 70513090000.0 (0.548 sec)
INFO:tensorflow:global_step/sec: 189.714
INFO:tensorflow:step = 7800, loss = 114337120000.0 (0.527 sec)
INFO:tensorflow:global_step/sec: 172.392
INFO:tensorflow:step = 7900, loss = 70231790000.0 (0.580 sec)
INFO:tensorflow:global_step/sec: 163.982
INFO:tensorflow:step = 8000, loss = 106731630000.0 (0.610 sec)
INFO:tensorflow:global_step/sec: 156.245
INFO:tensorflow:step = 8100, loss = 52576256000.0 (0.650 sec)
INFO:tensorflow:global_step/sec: 164.069
INFO:tensorflow:step = 8200, loss = 89418840000.0 (0.610 sec)
INFO:tensorflow:global_step/sec: 146.96
INFO:tensorflow:step = 8300, loss = 55530082000.0 (0.680 sec)
INFO:tensorflow:global_step/sec: 178.991
INFO:tensorflow:step = 8400, loss = 50860913000.0 (0.549 sec)
INFO:tensorflow:global_step/sec: 156.273
INFO:tensorflow:step = 8500, loss = 62489764000.0 (0.640 sec)
INFO:tensorflow:global_step/sec: 166.799
INFO:tensorflow:step = 8600, lo

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [144]:
pred_input = tf.estimator.inputs.pandas_input_fn(X_test, batch_size= 10, num_epochs= 1, shuffle= True)

In [145]:
predictions = dnn_model.predict(pred_input)

In [146]:
predictions = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\prash\AppData\Local\Temp\tmpjbvo2fl4\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [147]:
pred = []
for i in predictions:
     pred.append(i['predictions'])

In [148]:
from sklearn.metrics import mean_squared_error

In [149]:
mean_squared_error(y_test, pred)** 0.5

121363.53396429213